In [ ]:
import pandas as pd
import numpy as np
import PIL.Image as Image
import glob
from tqdm import tqdm_notebook
import cv2
import lasio

In [ ]:
w6406=lasio.read('./data/6406_3_2 grainsize.las')
print(w6406.keys())
depth = w6406['DEPT']
grain_size = w6406['GRAIN_SIZE']
grain_size

In [ ]:
w6406 = pd.DataFrame([depth, grain_size])
w6406 = w6406.T
w6406.head(5)
w6406.columns = ['depth', 'gs']




In [ ]:
#w6406 = w6406[['DEPT','GRAIN_SIZE']]
w64 = w6406[w6406>0].dropna()

In [ ]:
%matplotlib inline
#w64 = w64.set_index('depth')
w64.plot(figsize=(15,10))

In [ ]:
imgs = glob.glob('./data/well_6406_3_2_images/*.jpg')
train_df = pd.DataFrame(imgs)
train_df.columns=['imgName']

In [ ]:
train_df['img'] = [np.array(cv2.imread("{}".format(idx))) for idx in tqdm_notebook(train_df.imgName)]


In [ ]:
train_df['imgsize'] = train_df['img'].apply(lambda x: x.shape)

In [ ]:
print('max dimension', train_df['imgsize'].max())
print('min dimension', train_df['imgsize'].min())

In [ ]:
train_df['imgName']

In [ ]:
train_df['startDepth'] = train_df.imgName.str.replace('.jpg','').str.split('_').str[-2].astype(float)
train_df['endDepth'] = train_df.imgName.str.replace('.jpg','').str.split('_').str[-1].astype(float)
train_df['wellname']= train_df.imgName.str.split('/').str[2]

In [ ]:
sd = train_df.startDepth.values
ed = train_df.endDepth.values
gsd = w64.depth.values
gs = w64.gs.values

i,j = np.where((gsd[:,None] >= sd) & (gsd[:,None] <ed))
w64joined= pd.DataFrame(np.column_stack([w64.values[i],train_df.values[j]]), columns=w64.columns.append(train_df.columns))

In [ ]:
w64s = w64joined[['gs','depth','imgName','startDepth','endDepth','wellname']]

In [ ]:
w64joined_2000x260=w64joined
#Resize the image to 2000x260
w64joined_2000x260['img'] = w64joined_2000x260.apply(lambda x: cv2.resize(x['img'], (260, 2000)), axis=1)


In [ ]:
# crop the sides of the image to 244
w64joined_2000x244=w64joined_2000x260
w64joined_2000x244['img'] = w64joined_2000x260.apply(lambda x: x['img'][:, 18:242], axis=1)


In [ ]:
def processImg(xratio,img):
    xs = int(xratio * img.shape[0])
    xe = int((xratio + 0.2)*img.shape[0])
    newimg = img[xs:xe]
    return [newimg]
w64joined_2000x244['sdratio'] = (w64joined_2000x244['depth'] - w64joined_2000x244['startDepth'])/ \
( w64joined_2000x244['endDepth'] - w64joined_2000x244['startDepth']) - 0.15
w64joined_2000x244['imgp'] =  w64joined_2000x244[['sdratio','img']].apply(lambda x:processImg(x[0],x[1]),axis=1)

In [ ]:
w64joined['imgp'][0][0].shape

In [ ]:
w64s['gs'] = w64s['gs'].astype(float)
w64s['depth']=w64s['depth'].astype(float)
w64s.groupby(w64s.startDepth).aggregate({'gs':'mean','depth':'mean','imgName':'first','startDepth':'first'})

In [ ]:
aggregation_functions = {'gs': 'mean', 'depth': 'mean','imgName':'first', 'img': 'first','endDepth':'first','wellname':'first'}
w64g = w64joined.groupby(w64joined.startDepth).aggregate(aggregation_functions)

In [ ]:
w64joined_224x224=w64joined
#Resize the image to 2000x260
w64joined_224x224['img'] = w64joined_224x224.apply(lambda x: cv2.resize(x['img'], (224, 224)), axis=1)

In [ ]:
def saveImage(img,name):
    img2save = Image.fromarray(img)
    img2save.save(os.path.join('./data',name))

i=0
for d in w64joined_224x224.iterrows():
    i = i + 1
    print(d)
    name = d['wellname'] +"_" + str(d['startDepth']) + "_" + str(d['endDepth']) + '.png'
    print('name is ', name)
    saveImage(img[0],name)
    if i > 10:
        break
    